# Creating models with Jupyter notebooks

## Testing friction properties

The `pcg_gazebo_pkgs` includes also a simple interface to start Gazebo. 

It can be used to start Gazebo in the Python script or notebook, spawn models and interact with the simulation. 

In [1]:
try:
    import rospy
    ROS1_AVAILABLE = True
except ImportError:
    ROS1_AVAILABLE = False
    
if not ROS1_AVAILABLE:
    print('No ROS 1 available, source the ROS 1 setup.bash to run Gazebo')
else:
    # If there is a Gazebo instance running, you can spawn 
    # the box into the simulation
    from pcg_gazebo.task_manager import Server
    # First create a simulation server
    server = Server()
    # Create a simulation manager named default
    server.create_simulation('pcg-example')
    simulation = server.get_simulation('pcg-example')

In [2]:
if ROS1_AVAILABLE:
    # Run an instance of the empty.world scenario
    # This is equivalent to run
    #      roslaunch gazebo_ros empty_world.launch
    # with all default parameters
    simulation.create_gazebo_empty_world_task()
    # A task named 'gazebo' the added to the tasks list
    print('Tasks created: ', simulation.get_task_list())
    # But it is still not running
    print('Is Gazebo running: {}'.format(
        simulation.is_task_running('gazebo')))

Tasks created:  ['gazebo']
Is Gazebo running: False


In [3]:
if ROS1_AVAILABLE:
    # Run Gazebo
    simulation.run_all_tasks()

Once the simulation is running, an instance of the `GazeboProxy` object can be created to interact with it (e.g. spawn models).
If not specified otherwise, the simulation is created with a random port for Gazebo and ROS master to enable multiple instances of the simulation to run in parallel.

In [4]:
from pcg_gazebo.generators import WorldGenerator
import random
if ROS1_AVAILABLE:
    # Create a Gazebo proxy
    gazebo_proxy = simulation.get_gazebo_proxy()
    print('ROS configuration:')
    print(gazebo_proxy.ros_config)

ROS configuration:
ROS_MASTER_URI=http://localhost:16085, GAZEBO_MASTER_URI=http://localhost:27297


Create a single link box model.

In [5]:
from pcg_gazebo.simulation import SimulationModel

obj = SimulationModel('box')

# By changing the size, collision, visual and inertial 
# properties are already going to be updated
obj.add_cuboid_link(
    link_name='link',
    size=[0.8, 0.7, 0.9],
    mass=30)

# Print the initial state of a box in the model option
print(obj.to_sdf())

<model name="box">
  <pose frame="">0 0 0 0 -0 0</pose>
  <static>0</static>
  <self_collide>0</self_collide>
  <allow_auto_disable>0</allow_auto_disable>
  <link name="link">
    <collision name="collision">
      <geometry>
        <box>
          <size>0.8 0.7 0.9</size>
        </box>
      </geometry>
      <pose frame="">0 0 0 0 -0 0</pose>
    </collision>
    <visual name="visual">
      <geometry>
        <box>
          <size>0.8 0.7 0.9</size>
        </box>
      </geometry>
      <pose frame="">0 0 0 0 -0 0</pose>
      <cast_shadows>1</cast_shadows>
      <transparency>0.0</transparency>
    </visual>
    <inertial>
      <inertia>
        <ixx>3.25</ixx>
        <iyy>3.6250000000000004</iyy>
        <izz>2.825</izz>
        <ixy>0.0</ixy>
        <ixz>0.0</ixz>
        <iyz>0.0</iyz>
      </inertia>
      <pose frame="">0 0 0 0 -0 0</pose>
      <mass>30.0</mass>
    </inertial>
    <pose frame="">0 0 0 0 -0 0</pose>
  </link>
</model>



In [6]:
# Set default friction parameters
obj.links['link'].collisions[0].enable_property('friction')
print(obj.to_sdf('model'))

<model name="box">
  <pose frame="">0 0 0 0 -0 0</pose>
  <static>0</static>
  <self_collide>0</self_collide>
  <allow_auto_disable>0</allow_auto_disable>
  <link name="link">
    <collision name="collision">
      <geometry>
        <box>
          <size>0.8 0.7 0.9</size>
        </box>
      </geometry>
      <pose frame="">0 0 0 0 -0 0</pose>
    </collision>
    <visual name="visual">
      <geometry>
        <box>
          <size>0.8 0.7 0.9</size>
        </box>
      </geometry>
      <pose frame="">0 0 0 0 -0 0</pose>
      <cast_shadows>1</cast_shadows>
      <transparency>0.0</transparency>
    </visual>
    <inertial>
      <inertia>
        <ixx>3.25</ixx>
        <iyy>3.6250000000000004</iyy>
        <izz>2.825</izz>
        <ixy>0.0</ixy>
        <ixz>0.0</ixz>
        <iyz>0.0</iyz>
      </inertia>
      <pose frame="">0 0 0 0 -0 0</pose>
      <mass>30.0</mass>
    </inertial>
    <pose frame="">0 0 0 0 -0 0</pose>
  </link>
</model>



In [7]:
obj.links['link'].collisions[0].set_ode_friction_params(
    mu=0.9,
    mu2=0.5,
    slip1=0.3, 
    slip2=0.5,
    fdir1=[0, 0, 0]
)
print(obj.to_sdf('model'))

<model name="box">
  <pose frame="">0 0 0 0 -0 0</pose>
  <static>0</static>
  <self_collide>0</self_collide>
  <allow_auto_disable>0</allow_auto_disable>
  <link name="link">
    <collision name="collision">
      <geometry>
        <box>
          <size>0.8 0.7 0.9</size>
        </box>
      </geometry>
      <pose frame="">0 0 0 0 -0 0</pose>
      <surface>
        <friction>
          <ode>
            <mu>0.9</mu>
            <mu2>0.5</mu2>
            <fdir1>0 0 0</fdir1>
            <slip1>0.3</slip1>
            <slip2>0.5</slip2>
          </ode>
        </friction>
      </surface>
    </collision>
    <visual name="visual">
      <geometry>
        <box>
          <size>0.8 0.7 0.9</size>
        </box>
      </geometry>
      <pose frame="">0 0 0 0 -0 0</pose>
      <cast_shadows>1</cast_shadows>
      <transparency>0.0</transparency>
    </visual>
    <inertial>
      <inertia>
        <ixx>3.25</ixx>
        <iyy>3.6250000000000004</iyy>
        <izz>2.825</izz>
       

In [8]:
mu = [0.1, 0.3, 0.5, 0.7, 1.0] 
for i in range(len(mu)):
    print('\t - #{} mu = {}'.format(i, mu[i]))
    obj.links['link'].collisions[0].set_ode_friction_params(
        mu=mu[i],
        mu2=mu[i],
        slip1=0.0, 
        slip2=0.0)
    print(obj.links['link'].collisions[0].to_sdf())
    # Set a random XKCD color to each box
    obj.links['link'].visuals[0].set_xkcd_color()
    obj.name = 'box_mu_{}'.format(mu[i])
    obj.pose = [0, i, 2, 0, 0, 0]
    if ROS1_AVAILABLE:
        obj.spawn(gazebo_proxy=gazebo_proxy)

	 - #0 mu = 0.1
<collision name="collision">
  <geometry>
    <box>
      <size>0.8 0.7 0.9</size>
    </box>
  </geometry>
  <pose frame="">0 0 0 0 -0 0</pose>
  <surface>
    <friction>
      <ode>
        <mu>0.1</mu>
        <mu2>0.1</mu2>
        <fdir1>0 0 0</fdir1>
        <slip1>0.0</slip1>
        <slip2>0.0</slip2>
      </ode>
    </friction>
  </surface>
</collision>

	 - #1 mu = 0.3
<collision name="collision">
  <geometry>
    <box>
      <size>0.8 0.7 0.9</size>
    </box>
  </geometry>
  <pose frame="">0 0 0 0 -0 0</pose>
  <surface>
    <friction>
      <ode>
        <mu>0.3</mu>
        <mu2>0.3</mu2>
        <fdir1>0 0 0</fdir1>
        <slip1>0.0</slip1>
        <slip2>0.0</slip2>
      </ode>
    </friction>
  </surface>
</collision>

	 - #2 mu = 0.5
<collision name="collision">
  <geometry>
    <box>
      <size>0.8 0.7 0.9</size>
    </box>
  </geometry>
  <pose frame="">0 0 0 0 -0 0</pose>
  <surface>
    <friction>
      <ode>
        <mu>0.5</mu>
        <mu2>

![sim_boxes](images/sim_boxes.png)

Apply a force to each box to see the effects of the different friction coefficients.

In [9]:
if ROS1_AVAILABLE:
    for i in range(len(mu)):
        gazebo_proxy.apply_body_wrench(
            model_name='box_mu_{}'.format(mu[i]),
            link_name='link',
            force=[300, 0, 0],
            torque=[0, 0, 0],
            start_time=0,
            duration=2
            )

![sim_boxes_after_wrench](images/sim_boxes_after_wrench.png)

## Create a mobile base

Recreating the [Gazebo tutorial for a mobile base](http://gazebosim.org/tutorials/?tut=attach_gripper#MobileBase)

In [10]:
from pcg_gazebo.simulation import create_object

In [11]:
# Creating the main body of the chassis
chassis = create_object('box')
chassis.size = [2, 1, 0.3]
chassis.add_inertial(20)
chassis.visual.enable_property('material')
chassis.visual.set_xkcd_color()

print(chassis.to_sdf('link'))

<link name="box">
  <collision name="collision">
    <geometry>
      <box>
        <size>2 1 0.3</size>
      </box>
    </geometry>
    <pose frame="">0 0 0 0 -0 0</pose>
  </collision>
  <visual name="visual">
    <geometry>
      <box>
        <size>2 1 0.3</size>
      </box>
    </geometry>
    <material>
      <ambient>0.0941176 0.0196078 0.858824 1</ambient>
      <diffuse>0.0941176 0.0196078 0.858824 1</diffuse>
    </material>
    <pose frame="">0 0 0 0 -0 0</pose>
    <cast_shadows>1</cast_shadows>
    <transparency>0.0</transparency>
  </visual>
  <inertial>
    <inertia>
      <ixx>1.8166666666666667</ixx>
      <iyy>6.8166666666666655</iyy>
      <izz>8.333333333333332</izz>
      <ixy>0.0</ixy>
      <ixz>0.0</ixz>
      <iyz>0.0</iyz>
    </inertia>
    <pose frame="">0 0 0 0 -0 0</pose>
    <mass>20.0</mass>
  </inertial>
  <pose frame="">0 0 0 0 -0 0</pose>
</link>



In [12]:
# Creating the caster wheel
caster_wheel = create_object('sphere')
caster_wheel.radius = 0.125
caster_wheel.add_inertial(1)
caster_wheel.visual.enable_property('material')
caster_wheel.visual.set_xkcd_color()

# Setting friction parameters to zero
caster_wheel.collision.enable_property('friction')
caster_wheel.collision.set_ode_friction_params(
    mu=0.0,
    mu2=0.0,
    slip1=0, 
    slip2=0,
    fdir1=[0, 0, 0]
)

caster_wheel.collision.set_bullet_friction_params(
    friction=0.0, 
    friction2=0.0, 
    fdir1=[0, 0, 0], 
    rolling_friction=1
)

print(caster_wheel.to_sdf('link'))

<link name="sphere">
  <collision name="collision">
    <geometry>
      <sphere>
        <radius>0.125</radius>
      </sphere>
    </geometry>
    <pose frame="">0 0 0 0 -0 0</pose>
    <surface>
      <friction>
        <ode>
          <mu>0.0</mu>
          <mu2>0.0</mu2>
          <fdir1>0 0 0</fdir1>
          <slip1>0.0</slip1>
          <slip2>0.0</slip2>
        </ode>
        <bullet>
          <friction>0.0</friction>
          <friction2>0.0</friction2>
          <rolling_friction>1.0</rolling_friction>
          <fdir1>0 0 0</fdir1>
        </bullet>
      </friction>
    </surface>
  </collision>
  <visual name="visual">
    <geometry>
      <sphere>
        <radius>0.125</radius>
      </sphere>
    </geometry>
    <material>
      <ambient>0.0156863 0.85098 1 1</ambient>
      <diffuse>0.0156863 0.85098 1 1</diffuse>
    </material>
    <pose frame="">0 0 0 0 -0 0</pose>
    <cast_shadows>1</cast_shadows>
    <transparency>0.0</transparency>
  </visual>
  <inertial>
   

In [13]:
right_wheel = create_object('cylinder')
right_wheel.pose = [-0.8, -0.5 - 0.025, -0.125, 0, 1.5707, 1.5707]
right_wheel.radius = 0.125
right_wheel.length = 0.05
right_wheel.add_inertial(1)
right_wheel.visual.enable_property('material')
right_wheel.visual.set_xkcd_color()

left_wheel = create_object('cylinder')
left_wheel.pose = [-0.8, 0.5 + 0.025, -0.125, 0, 1.5707, 1.5707]
left_wheel.radius = 0.125
left_wheel.length = 0.05
left_wheel.add_inertial(1)
left_wheel.visual.enable_property('material')
left_wheel.visual.set_xkcd_color()

print(right_wheel.to_sdf('link'))

<link name="cylinder">
  <collision name="collision">
    <geometry>
      <cylinder>
        <radius>0.125</radius>
        <length>0.05</length>
      </cylinder>
    </geometry>
    <pose frame="">0 0 0 0 -0 0</pose>
  </collision>
  <visual name="visual">
    <geometry>
      <cylinder>
        <radius>0.125</radius>
        <length>0.05</length>
      </cylinder>
    </geometry>
    <material>
      <ambient>0.509804 0.372549 0.529412 1</ambient>
      <diffuse>0.509804 0.372549 0.529412 1</diffuse>
    </material>
    <pose frame="">0 0 0 0 -0 0</pose>
    <cast_shadows>1</cast_shadows>
    <transparency>0.0</transparency>
  </visual>
  <inertial>
    <inertia>
      <ixx>0.004114583333333333</ixx>
      <iyy>0.004114583333333333</iyy>
      <izz>0.0078125</izz>
      <ixy>0.0</ixy>
      <ixz>0.0</ixz>
      <iyz>0.0</iyz>
    </inertia>
    <pose frame="">0 0 0 0 -0 0</pose>
    <mass>1.0</mass>
  </inertial>
  <pose frame="">-0.8 -0.525 -0.125 5.76279e-13 1.5707 1.5707</pose>


In [14]:
# Assembling the robot mobile base
mobile_base = SimulationModel('mobile_base')

# Adding chassis
mobile_base.add_link('chassis', chassis)

# Adding caster wheel
caster_wheel.pose = [0.8, 0, -0.125, 0, 0, 0]
mobile_base.add_link('caster_wheel', caster_wheel)
mobile_base.add_joint('caster_wheel_joint', parent='chassis', child='caster_wheel', joint_type='fixed')

# Adding left wheel
mobile_base.add_link('left_wheel', left_wheel)
mobile_base.add_joint('left_wheel_joint', parent='chassis', child='left_wheel', joint_type='revolute')

# Adding right wheel
mobile_base.add_link('right_wheel', right_wheel)
mobile_base.add_joint('right_wheel_joint', parent='chassis', child='right_wheel', joint_type='revolute')

print(mobile_base.to_sdf())

<model name="mobile_base">
  <pose frame="">0 0 0 0 -0 0</pose>
  <static>0</static>
  <self_collide>0</self_collide>
  <allow_auto_disable>0</allow_auto_disable>
  <link name="chassis">
    <collision name="collision">
      <geometry>
        <box>
          <size>2 1 0.3</size>
        </box>
      </geometry>
      <pose frame="">0 0 0 0 -0 0</pose>
    </collision>
    <visual name="visual">
      <geometry>
        <box>
          <size>2 1 0.3</size>
        </box>
      </geometry>
      <material>
        <ambient>0.0941176 0.0196078 0.858824 1</ambient>
        <diffuse>0.0941176 0.0196078 0.858824 1</diffuse>
      </material>
      <pose frame="">0 0 0 0 -0 0</pose>
      <cast_shadows>1</cast_shadows>
      <transparency>0.0</transparency>
    </visual>
    <inertial>
      <inertia>
        <ixx>1.8166666666666667</ixx>
        <iyy>6.8166666666666655</iyy>
        <izz>8.333333333333332</izz>
        <ixy>0.0</ixy>
        <ixz>0.0</ixz>
        <iyz>0.0</iyz>
      </in

In [15]:
print(mobile_base.to_urdf())

<robot name="mobile_base">
  <link name="chassis">
    <inertial>
      <mass value="20.0"/>
      <origin xyz="0 0 0" rpy="0.0 -0.0 0.0"/>
      <inertia ixx="1.8166666666666667" iyy="6.8166666666666655" izz="8.333333333333332" ixy="0.0" ixz="0.0" iyz="0.0"/>
    </inertial>
    <visual name="visual">
      <origin xyz="0 0 0" rpy="0.0 -0.0 0.0"/>
      <geometry>
        <box size="2 1 0.3"/>
      </geometry>
      <material name="">
        <color rgba="0.09411764705882353 0.0196078431372549 0.8588235294117647 1"/>
      </material>
    </visual>
    <collision name="collision">
      <origin xyz="0 0 0" rpy="0.0 -0.0 0.0"/>
      <geometry>
        <box size="2 1 0.3"/>
      </geometry>
    </collision>
  </link>
  <link name="caster_wheel">
    <inertial>
      <mass value="1.0"/>
      <origin xyz="0 0 0" rpy="0.0 -0.0 0.0"/>
      <inertia ixx="0.00625" iyy="0.00625" izz="0.00625" ixy="0.0" ixz="0.0" iyz="0.0"/>
    </inertial>
    <visual name="visual">
      <origin xyz="0 0

In [16]:
if ROS1_AVAILABLE:
    mobile_base.spawn(
        gazebo_proxy=gazebo_proxy,
        pos=[0, 0, 0.5])

True

![sim_mobile_base](images/sim_mobile_base.png)

## Add sensors to mobile base

In [17]:
# Add IMU sensor
mobile_base.add_imu_sensor(
    parent='chassis',
    add_visual=True, 
    add_collision=True, 
    visualize=True,
    mass=0.01,
    size=[0.1, 0.1, 0.1],
    topic='/imu',
    link_shape='cuboid',
    link_name='imu_link')

# Add contact sensor
mobile_base.add_contact_sensor(
    pose=[1, 0, 0, 0, 0, 0],
    parent='chassis',
    add_visual=True, 
    add_collision=True, 
    add_ros_plugin=True,
    mass=0.01,
    radius=0.05,
    length=0.1,
    link_shape='cylindrical',
    link_name='contact_link')

# Add camera sensor
mobile_base.add_camera_sensor(
    pose=[1, 0, 0.2, 0, 0, 0],
    parent='chassis',
    add_visual=True, 
    add_collision=True, 
    add_ros_plugin=True,
    visualize=True,
    mass=0.01,
    size=[0.1, 0.1, 0.1],
    link_shape='cuboid',
    link_name='camera_link')

True

In [18]:
print(mobile_base.to_sdf())

<model name="mobile_base">
  <pose frame="">0 0 0 0 -0 0</pose>
  <static>0</static>
  <self_collide>0</self_collide>
  <allow_auto_disable>0</allow_auto_disable>
  <link name="chassis">
    <collision name="collision">
      <geometry>
        <box>
          <size>2 1 0.3</size>
        </box>
      </geometry>
      <pose frame="">0 0 0 0 -0 0</pose>
    </collision>
    <visual name="visual">
      <geometry>
        <box>
          <size>2 1 0.3</size>
        </box>
      </geometry>
      <material>
        <ambient>0.0941176 0.0196078 0.858824 1</ambient>
        <diffuse>0.0941176 0.0196078 0.858824 1</diffuse>
      </material>
      <pose frame="">0 0 0 0 -0 0</pose>
      <cast_shadows>1</cast_shadows>
      <transparency>0.0</transparency>
    </visual>
    <inertial>
      <inertia>
        <ixx>1.8166666666666667</ixx>
        <iyy>6.8166666666666655</iyy>
        <izz>8.333333333333332</izz>
        <ixy>0.0</ixy>
        <ixz>0.0</ixz>
        <iyz>0.0</iyz>
      </in

In [19]:
if ROS1_AVAILABLE:
    mobile_base.spawn(
        gazebo_proxy=gazebo_proxy,
        robot_namespace='mobile_base_with_sensor',
        pos=[0, 2.0, 0.5])

True

![sim_modified_mobile_base](images/sim_modified_mobile_base.png)

In [20]:
if ROS1_AVAILABLE:
    simulation.kill_all_tasks()

## Export generated model

Simulation models can be exported as an SDF, URDF or as a static Gazebo model.

In case the Gazebo model is exported, it can be stored in a provided folder path or it will be stored per default in `$HOME/.gazebo/models`.

In [21]:
model_path = mobile_base.to_gazebo_model(overwrite=True)

In [22]:
import os
print('Gazebo model folder: ', model_path)
print(os.listdir(model_path))

Gazebo model folder:  /home/mam0box/.gazebo/models/mobile_base
['model.config', 'model.sdf', 'model.urdf']


In [23]:
mobile_base.to_urdf().export_xml(os.path.join(model_path, 'model.urdf'))
print(os.listdir(model_path))

['model.config', 'model.sdf', 'model.urdf']
